# Теория

<абзац про присяжных>

### где сегодня используются ансамбли
- Везде, где можно применять классические алгоритмы (ансамбли дают более точные результаты)
- Поисковые системы (ранжирование реультатов)
- Компьютерное зрение (распознавание объектов)

### Идея
Если взять несколько методов научить их последовательно исправлять ошибки друг друга, качество такой системы будет выше, чем каждого из методов по отдельности.

Лучше, если алгоритмы неустойчивы к аномалиям в данных: поэтому для ансамблей берут Регрессию и Деревья Решений.

### Как собрать ансамбль?
- ансамбль собирают из supervised-алгоритмов, потому что важно знать, в чем/где ошибаются модели
- в плане последовательности можно собрать как угодно, но опытным путем нашлись три способа: *стэкинг*, *бэггинг* и *бустинг*


## Stacking

Идея: на одних и тех же данных обучаем несколько разных алгоритмов (например, классификации). Передаем реультаты финальному, он принимает решение (обычно это регрессия).

<img alt="" width="900" height="600" src="https://miro.medium.com/max/1892/0*GHYCJIjkkrP5ZgPh.png">

[ссылка на картинку](https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de)

*С добавлением новых моделей в ансамбль, мы не повысим качество предсказаний*

## Bagging

Идея: несколько раз тренируем один и тот же алгоритм на разных подвыборках из данных. В результате усредняем ответы и определяем финальный. 
<img alt="" width="700" height="400" src="https://miro.medium.com/max/1920/1*DFHUbdz6EyOuMYP4pDnFlw.jpeg">

[ссылка на картинку](https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de)

Самый популярный бэггинг - это [Random Forest](https://ru.wikipedia.org/wiki/Random_forest) (набор решающих деревьев)<br>
Бэггинг -- эффективный метод, если у вас небольшой датасет.<br>
[Чем бэггинг отличается от кросс-валидации?](https://www.kaggle.com/questions-and-answers/120778)

## Boosting

Идея: обучаем алгоритмы последовательно, каждый следующий уделяет внимание ошибкам предыдущего. Продолжаем, пока метрики не станут хорошими

<img alt="Boosting procedure" src="https://pluralsight2.imgix.net/guides/a9a5ff4e-b617-4afe-b27b-d96793defa87_6.jpg">

[ссылка на картинку](https://www.pluralsight.com/guides/ensemble-methods:-bagging-versus-boosting)

[статья про популярные типы бустинга](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db)

### разница между бэггингом и бустингом

<img alt="Bagging, Boosting" width="700" height="400" src="https://pluralsight2.imgix.net/guides/81232a78-2e99-4ccc-ba8e-8cd873625fdf_2.jpg">

**Когда что использовать?**
- Бэггинг, если у данных высокая дисперсия
- Бустинг, если данные неравномерно распределены

**Bagging VS Boosting**

Нет одноначного ответа, что лучше, это зависит от задачи и данных. Если модель хорошо работает на обучающих данных, но плохо на тестовых, то в данных может быть высокая диспрерсия. Если модель плохо работает на обучающих данных, стоит проверить распределение классов/параметров.
Предварительно стоит проаналиировать данные, затем попробовать несколько моделей, постепенно изменяя параметры.

# Практика

Грузим датасет Wine. Будем предсказывать происхождение вина по его химическим характеристикам.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## подготовим данные

In [2]:
wine = load_wine()

In [82]:
data = pd.DataFrame(wine['data'], columns=wine['feature_names'])
target = wine.target

In [85]:
print(data.shape)
data.head(3)

(178, 13)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0


In [88]:
print(target.shape)

target

(178,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [6]:
# разделение на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=25)

## Bagging

сначала обучим одно решающее дерево

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [8]:
tree_clf = DecisionTreeClassifier()

In [9]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [72]:
tree_y_pred = tree_clf.predict(X_test)

In [73]:
print(classification_report(y_test, tree_y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        17
           1       0.96      0.88      0.92        25
           2       0.86      1.00      0.92        12

    accuracy                           0.93        54
   macro avg       0.92      0.94      0.93        54
weighted avg       0.93      0.93      0.93        54



теперь обучим набор деревьев: Random Forest

In [119]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier(n_estimators=3, max_depth=5, verbose=5)

используемые параметры модели: 
- n_estimators : число деревьев в ансамбле (лесе)
- max_depth : глубина дерева 
- verbose : пояснения о процессе обучения

In [120]:
forest.fit(X_train, y_train)

building tree 1 of 3
building tree 2 of 3
building tree 3 of 3


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


RandomForestClassifier(max_depth=5, n_estimators=3, verbose=5)

In [121]:
forest_y_pred = forest.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [122]:
print(classification_report(y_test, forest_y_pred))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91        17
           1       0.89      0.96      0.92        25
           2       1.00      0.92      0.96        12

    accuracy                           0.93        54
   macro avg       0.94      0.92      0.93        54
weighted avg       0.93      0.93      0.93        54



## Boosting

In [32]:
from sklearn.ensemble import 

In [161]:
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=10, verbose=5)

используемые параметры [модели](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier):
- loss : The loss function to be optimized. ‘deviance’ refers to deviance (= logistic regression) for classification with probabilistic outputs. For loss ‘exponential’ gradient boosting recovers the AdaBoost algorithm.
- n_estimators : The number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.
- random_state : Controls the random seed given to each Tree estimator at each boosting iteration. In addition, it controls the random permutation of the features at each split (see Notes for more details). It also controls the random spliting of the training data to obtain a validation set if n_iter_no_change is not None. Pass an int for reproducible output across multiple function calls. See Glossary.
- verbose : Enable verbose output. If 1 then it prints progress and performance once in a while (the more trees the lower the frequency). If greater than 1 then it prints progress and performance for every tree.


In [162]:
gb_clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.9061            0.78s
         2           0.7605            0.63s
         3           0.6443            0.64s
         4           0.5493            0.59s
         5           0.4708            0.57s
         6           0.4051            0.55s
         7           0.3497            0.52s
         8           0.3026            0.51s
         9           0.2628            0.49s
        10           0.2282            0.51s
        11           0.1988            0.51s
        12           0.1735            0.51s
        13           0.1519            0.51s
        14           0.1331            0.52s
        15           0.1162            0.52s
        16           0.1016            0.52s
        17           0.0889            0.52s
        18           0.0778            0.52s
        19           0.0682            0.51s
        20           0.0598            0.51s
        21           0.0525            0.51s
        2

GradientBoostingClassifier(random_state=10, verbose=5)

In [163]:
gb_y_pred = gb_clf.predict(X_test)

In [164]:
print(classification_report(y_test, gb_y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91        17
           1       0.96      0.92      0.94        25
           2       1.00      1.00      1.00        12

    accuracy                           0.94        54
   macro avg       0.95      0.95      0.95        54
weighted avg       0.95      0.94      0.94        54



Сколько деревьев мы обучили? Сколько было бы достаточно для получения текущего качества?
(подсказка: посмотрите на loss и n_estimators)

**Early stopping** - метод, при котором перестаем обучаться, если ошибка не уменьшается/ уменьшается незначительно. За это отвечают следующие параметры модели:

- tol : tolerance for the early stopping, оставим значение дефолтным
- n_iter_no_change :  останавливаем ли обучение, если validation score больше не увеличивается<br>
(если функция ошибки(train loss) не уменьшается хотя бы на tol в течение n_iter_no_change итераций, то прекращаем обучение)

In [165]:
gb_clf_es = GradientBoostingClassifier(n_iter_no_change=5, verbose=10)

In [125]:
gb_clf_es.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.9055            0.91s
         2           0.7600            0.88s
         3           0.6436            1.00s
         4           0.5488            0.90s
         5           0.4703            0.82s
         6           0.4035            0.83s
         7           0.3472            0.82s
         8           0.2996            0.79s
         9           0.2590            0.76s
        10           0.2244            0.75s
        11           0.1946            0.74s
        12           0.1690            0.73s
        13           0.1469            0.70s
        14           0.1278            0.68s
        15           0.1113            0.65s
        16           0.0970            0.63s
        17           0.0846            0.61s
        18           0.0738            0.60s
        19           0.0644            0.58s
        20           0.0562            0.57s
        21           0.0491            0.56s
        2

GradientBoostingClassifier(n_iter_no_change=5, verbose=10)

In [130]:
es_y_pred = gb_clf_es.predict(X_test)

In [131]:
print(classification_report(y_test, es_y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91        17
           1       0.96      0.92      0.94        25
           2       1.00      1.00      1.00        12

    accuracy                           0.94        54
   macro avg       0.95      0.95      0.95        54
weighted avg       0.95      0.94      0.94        54



Сравнивая метрики, какой алгоритм оказался лучше?

Можно посмотреть коэффициенты важности признаков для разных моделей.

In [154]:
# Decision Tree Classifier
pd.DataFrame(tree_clf.feature_importances_).nlargest(5,0)

,0
12,0.396020
6,0.387435
9,0.105140
4,0.081013
7,0.018235


In [153]:
# Random Forest
pd.DataFrame(forest.feature_importances_).nlargest(5,0)

,0
12,0.171758
5,0.145654
10,0.138226
6,0.112278
9,0.103266


In [152]:
# GradientBoostingClassifier
pd.DataFrame(gb_clf.feature_importances_).nlargest(5,0)

,0
6,0.300190
9,0.296082
12,0.268266
4,0.063681
2,0.031499


In [151]:
# GradientBoostingClassifier + Early stopping
pd.DataFrame(gb_clf_es.feature_importances_).nlargest(5,0)

,0
6,0.309203
9,0.289139
12,0.262669
4,0.072204
2,0.019162
